In [ ]:
import numpy as np
import pandas as pd


In [ ]:
df = pd.read_csv('/Users/mannes/thesis/correlation_evaluation/Qwen7B_math_run.csv')

# df = pd.read_csv('/Users/mannes/thesis/complex_task_gen/output/output/df_eval_daniil.csv')
df_corr = pd.read_csv('/Users/mannes/thesis/correlation_evaluation/correlation_table.csv')

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr

# Assuming df_eval is your DataFrame
alphas = np.linspace(-1, 2, 301)
best_alpha, best_corr = None, -1

for alpha in alphas:
    df["score"] = df["logp_sum"] / (df["n_tokens"] ** alpha)
    rho, _ = spearmanr(df["score"], df["pass1_int"])  # assuming pass1_int is 1/0 pass label
    if abs(rho) > best_corr:
        best_alpha, best_corr = alpha, abs(rho)

print(f"Best alpha: {best_alpha:.3f}, correlation: {best_corr:.3f}")

In [ ]:
df["score_alpha"] = df["logp_sum"] / (df["n_tokens"] ** -0.610)

from scipy.stats import pearsonr, spearmanr

spearman_corr = spearmanr(df["score_alpha"], df["pass1_int"])
pearson_corr = pearsonr(df["score_alpha"], df["pass1_int"])

print("Spearman:", spearman_corr.correlation, "p-value:", spearman_corr.pvalue)
print("Pearson:", pearson_corr.correlation, "p-value:", pearson_corr.pvalue)


In [ ]:
import matplotlib.pyplot as plt

thresholds = np.linspace(df["score_alpha"].min(), df["score_alpha"].max(), 100)
pass_rates = [(df["score_alpha"] > t).mean() for t in thresholds]
accuracies = [df[df["score_alpha"] > t]["pass1_int"].mean() for t in thresholds]

plt.plot(pass_rates, accuracies)
plt.xlabel("Proportion of samples retained")
plt.ylabel("Accuracy")
plt.title("Accuracy vs coverage curve")
plt.grid()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import spearmanr, ttest_ind

# Plot correlation vs alpha
alphas = np.linspace(-1, 2, 301)
correlations = []

for alpha in alphas:
    df["score"] = df["logp_sum"] / (df["n_tokens"] ** alpha)
    rho, _ = spearmanr(df["score"], df["pass1_int"])
    correlations.append(rho)

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(alphas, correlations)
plt.axvline(x=-0.610, color='r', linestyle='--', label=f'Optimal α = -0.610')
plt.xlabel('Alpha value')
plt.ylabel('Spearman correlation with pass@1')
plt.title('Correlation vs Alpha Parameter')
plt.grid(True)
plt.legend()
plt.show()

# Calculate scores with optimal alpha
df["score_optimal"] = df["logp_sum"] / (df["n_tokens"] ** -0.610)

# Group by correct/incorrect
correct_scores = df[df["pass1_int"] == 1]["score_optimal"]
incorrect_scores = df[df["pass1_int"] == 0]["score_optimal"]

# Statistical summary
summary = pd.DataFrame({
    'Group': ['Correct', 'Incorrect'],
    'Count': [len(correct_scores), len(incorrect_scores)],
    'Mean': [correct_scores.mean(), incorrect_scores.mean()],
    'Std': [correct_scores.std(), incorrect_scores.std()],
    'Min': [correct_scores.min(), incorrect_scores.min()],
    'Max': [correct_scores.max(), incorrect_scores.max()]
})

print("Summary statistics:")
print(summary)

# Run t-test
t_stat, p_value = ttest_ind(correct_scores, incorrect_scores, equal_var=False)
print(f"\nT-test results: t-statistic = {t_stat:.4f}, p-value = {p_value:.8f}")
if p_value < 0.05:
    print("The mean scores are statistically significantly different.")

# Visualize the distributions
# Visualize the distributions
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x="score_optimal", hue="pass1_int",
             kde=True, common_norm=False, stat="density",
             palette=["green", "red"])
# Update legend labels after plotting
plt.legend(title='Sample Status', labels=["Incorrect", "Correct"])
plt.title('Distribution of Scores for Correct vs Incorrect Samples')
plt.xlabel('Score (logp_sum / (n_tokens ** -0.610))')
plt.ylabel('Density')
plt.show()

In [ ]:
# df_corr = pd.read_csv('/Users/mannes/thesis/complex_task_gen/output/output/df_corr.csv')

In [ ]:
# Few ideas:
# Try log of tokens
# If we want for sanity check, try to run 7b model if possible -> Run entire pipeline with larger model

# Try to run with 7b model -> This is the correlation between metric x and y
# Then forget about this
# Use Pass@1 as novelty -> Take target model, when I generate, I will use the pass@1 as novelty
# Run model 4 times on -> calculate probability of the model being correct -> Then average this as the target model and give it as a reward -> more wasteful  but will still be good
# For 1.5B it will be quick enoguh
# Run target model on VLLM --> Generate 8 solutions for given task , average the  'likelihood' of the model being correct
# Style of the solution can be

In [ ]:
df_corr

In [ ]:
df

In [ ]:
# Run for 7B Model --> check that daniil is not insane
# Update novelty score